In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import Workbook

In [3]:
list_category = pd.read_csv("Data\Origin_DATA\list_category.csv")
item_cz = pd.read_csv("Data\Origin_DATA\item_cz.csv")
material_attributes_cz = pd.read_csv("Data\Origin_DATA\material_attributes_cz.csv")

In [5]:
rows = 5e6
product_views_cz = pd.read_csv("Data\product_views_cz.csv",nrows = rows)

In [ ]:
chunksize = 5e6
i = 0
pd.read_csv("Data\Origin_DATA\product_views_cz.csv",nrows = 0).to_csv('product_views_cz_LY.csv',index=False)
for chunk in pd.read_csv("Data\Origin_DATA\product_views_cz.csv",chunksize = chunksize):
    i += 1
    print(i)
    chunk[chunk['date'] > '2019-10-00'].to_csv('product_views_cz_LY.csv', mode='a', header=False,index=False)
print('origin count of rows: '+str(i*5e6))

In [ ]:
chunksize = 5e2
list_category_MACZ = pd.DataFrame()
for chunk in pd.read_csv("Data\list_category.csv",chunksize = chunksize):
    chunk[chunk['shop'] == 'MACZ'].to_csv('out.csv', mode='a', header=False)

product_views_cz zabírá moc velkou paměť, lepší průběžně ukládat

In [16]:
print(len(set(item_cz['sid'])))
print(len(set(item_cz['grouping_sid'])))

10077099
5054473


In [17]:
rows = 5e6
cz_item_category = pd.read_csv("Data\cz_item_category.csv",nrows = rows)

In [46]:
print(set(cz_item_category[cz_item_category['item_id'] == 8061405]['category_id']))
print(cz_item_category[(cz_item_category['item_id'] == 8061405) & (cz_item_category['is_main'] == 't')])

{3677194, 237, 1051598, 1842, 1017621, 1140857}
         item_id  category_id                    ods_created is_main
748757   8061405          237  2020-07-26 02:04:11.522873+00       t
789751   8061405          237  2020-07-21 07:50:53.932572+00       t
891311   8061405          237  2020-09-04 04:52:15.748332+00       t
1052189  8061405          237  2020-08-03 15:52:15.975838+00       t
1192159  8061405          237  2019-05-09 10:32:32.255025+00       t
1951701  8061405          237  2020-08-25 00:30:51.047528+00       t
2031887  8061405          237    2020-08-28 05:48:24.9943+00       t
3056757  8061405          237  2020-07-03 04:52:53.116226+00       t
3561783  8061405          237   2020-07-06 14:08:33.38948+00       t
3882526  8061405          237  2020-08-03 22:27:36.423737+00       t
3904855  8061405          237  2020-09-02 00:40:42.915108+00       t
4138200  8061405          237  2020-08-28 07:36:42.755617+00       t


.drop_duplicates() == DISTINCT 

In [48]:
cz_item_category[['item_id','category_id','is_main']].drop_duplicates()

,item_id,category_id,is_main
0,8061405,1051598,f
1,7807990,5217670,f
2,7746417,237,f
3,656838977,49931,f
4,158574663,1480415,f
...,...,...,...
4999927,661384444,237,f
4999963,1414043127,1150564,f
4999965,1398286037,1140855,f
4999968,581548995,237,f


In [3]:
chunksize = 5e6
i = 0
pd.read_csv("Data\Origin_DATA\cz_item_category.csv",nrows = 0)[['item_id','category_id','is_main']].to_csv('cz_item_category_DISTINCT.csv',index=False)
for chunk in pd.read_csv("Data\Origin_DATA\cz_item_category.csv",chunksize = chunksize):
    i += 1
    print(i)
    chunk[['item_id','category_id','is_main']].drop_duplicates().to_csv('cz_item_category_DISTINCT.csv', mode='a', header=False,index=False)
print('origin count of rows: '+str(i*5e6))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
origin count of rows: 460000000.0


In [6]:
list_category = pd.read_csv("Data\list_category.csv")
len(list_category[list_category['shop'] == 'MACZ'])

17359

In [5]:
list_category_MACZ = list_category[list_category['shop']=='MACZ']

In [7]:
print(len(list_category_MACZ['id']))
print(len(set(list_category_MACZ['id'])))

17359
7704


In [6]:
CATEGORY_TREE = list()
for row in range(len(list_category_MACZ)):
    A = list()
    j = 0
    a = 0
    for i in list_category_MACZ['path'].iloc[row]:
        length = len(str(list_category_MACZ['path'].iloc[row]))
        if i == '>':
            A.append(str(list_category_MACZ['path'].iloc[row])[a:j].strip())
            a = j+1
            if '>' not in str(list_category_MACZ['path'].iloc[row])[j+1:-1]:
                A.append(str(list_category_MACZ['path'].iloc[row])[j+1:-1].strip()+str(list_category_MACZ['path'].iloc[row])[-1])
                A.append(str(list_category_MACZ['id'].iloc[row]))
                break
        j += 1
    if A == []:
        A.append(list_category_MACZ['path'].iloc[row])
        A.append(str(list_category_MACZ['id'].iloc[row]))            
    CATEGORY_TREE.append(A)

In [103]:
maximum = 3
row = 0
for i in range(len(CATEGORY_TREE)):
    row += 1
    if len(CATEGORY_TREE[i])>maximum:
        maximum = len(CATEGORY_TREE[i])
        max_row = row
print(maximum)
max_row

8


223

In [105]:
number = 2468
print(CATEGORY_TREE[number])
print(list_category_MACZ['path'].iloc[number])

['Sport, outdoor', 'Zimní sporty', 'Sjezdové lyžování', 'Přilby', 'Pánské', '1681']
Sport, outdoor > Zimní sporty > Sjezdové lyžování > Přilby > Pánské


In [7]:
CATEGORY_TREE_norm = list()
chyba = 0
for row in CATEGORY_TREE:
    chyba+=1
    if len(row)<8:
        A = list()
        for j in range(len(row)-1):
            A.append(row[j])
        for j in range(8-len(row)):
            A.append(None)
        A.append(row[len(row)-1])
        CATEGORY_TREE_norm.append(A)
    else:
        CATEGORY_TREE_norm.append(row)

In [8]:
category_tree = pd.DataFrame(CATEGORY_TREE_norm)

In [9]:
category = category_tree.drop_duplicates()

In [10]:
category.columns = ['category_0','category_1','category_2','category_3','category_4','category_5','category_6','category_id']

In [ ]:
category.to_csv('category_tree.csv')

In [18]:
B = []
for cat_0 in set(category['category_0']):
    B.append([cat_0,len(category[category['category_0']==cat_0])])
B = pd.DataFrame(B)
B.to_excel('B.xlsx')

In [12]:
category.to_excel('category_tree.xlsx')

d:\video_prednasky_zim20_21\das\venv\das_1\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,22,2020-07-11,26765943,14,1018700,1
0,55,2020-07-02,25820914,2,100004359972,1
1,56,2020-07-02,25820914,3,100017552468,1
2,235,2020-04-02,14723304,172,100011803451,1
3,236,2020-04-02,14723304,173,100011612041,1
4,237,2020-04-08,14723304,174,100016624290,1
...,...,...,...,...,...,...
78058195,420387584,2020-05-31,36134412,1,1413786,1
78058196,420387609,2020-04-24,59841901,1,1344238,1
78058197,420387618,2020-09-25,28468748,1,1273124,1
78058198,420387620,2020-08-18,49759601,2,579250,1
